# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 23 2022 8:30AM,247104,10,0086002651,ISDIN Corporation,Released
1,Sep 23 2022 8:30AM,247104,10,0086002959,ISDIN Corporation,Released
2,Sep 23 2022 8:30AM,247104,10,0086002964,ISDIN Corporation,Released
3,Sep 23 2022 8:30AM,247104,10,0086002976,ISDIN Corporation,Released
4,Sep 23 2022 8:30AM,247104,10,0086003140,ISDIN Corporation,Released
5,Sep 23 2022 8:30AM,247104,10,0086003151,ISDIN Corporation,Released
6,Sep 23 2022 8:30AM,247104,10,0086003154,ISDIN Corporation,Released
7,Sep 23 2022 8:30AM,247104,10,0086003165,ISDIN Corporation,Released
8,Sep 23 2022 8:30AM,247104,10,0086003167,ISDIN Corporation,Released
9,Sep 23 2022 8:30AM,247104,10,0086003166,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,247106,Released,10
26,247107,Released,1
27,247109,Released,2
28,247110,Released,3
29,247111,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247106,NaN,NaN,10.0
247107,NaN,NaN,1.0
247109,NaN,NaN,2.0
247110,NaN,NaN,3.0
247111,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246998,0.0,1.0,0.0
246999,0.0,0.0,1.0
247005,8.0,1.0,2.0
247031,0.0,0.0,1.0
247048,0.0,1.0,34.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246998,0,1,0
246999,0,0,1
247005,8,1,2
247031,0,0,1
247048,0,1,34


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246998,0,1,0
1,246999,0,0,1
2,247005,8,1,2
3,247031,0,0,1
4,247048,0,1,34


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246998,,1,
1,246999,,,1
2,247005,8,1,2
3,247031,,,1
4,247048,,1,34


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 23 2022 8:30AM,247104,10,ISDIN Corporation
11,Sep 23 2022 8:30AM,247105,10,ISDIN Corporation
15,Sep 23 2022 8:30AM,247106,10,ISDIN Corporation
25,Sep 23 2022 8:30AM,247067,10,Eywa Pharma Inc.
26,Sep 23 2022 8:30AM,247109,10,ISDIN Corporation
28,Sep 22 2022 4:47PM,247111,16,Sartorius Lab Products and Service
30,Sep 22 2022 4:47PM,247110,16,Sartorius Bioprocess Solutions
33,Sep 22 2022 4:22PM,247107,16,"Emersa Waterbox, LLC"
34,Sep 22 2022 4:11PM,247103,19,Eywa Pharma Inc.
37,Sep 22 2022 3:41PM,247102,12,Hush Hush


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 23 2022 8:30AM,247104,10,ISDIN Corporation,,,11
1,Sep 23 2022 8:30AM,247105,10,ISDIN Corporation,,,4
2,Sep 23 2022 8:30AM,247106,10,ISDIN Corporation,,,10
3,Sep 23 2022 8:30AM,247067,10,Eywa Pharma Inc.,,,1
4,Sep 23 2022 8:30AM,247109,10,ISDIN Corporation,,,2
5,Sep 22 2022 4:47PM,247111,16,Sartorius Lab Products and Service,,,2
6,Sep 22 2022 4:47PM,247110,16,Sartorius Bioprocess Solutions,,,3
7,Sep 22 2022 4:22PM,247107,16,"Emersa Waterbox, LLC",,,1
8,Sep 22 2022 4:11PM,247103,19,Eywa Pharma Inc.,,,3
9,Sep 22 2022 3:41PM,247102,12,Hush Hush,,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 23 2022 8:30AM,247104,10,ISDIN Corporation,11,,
1,Sep 23 2022 8:30AM,247105,10,ISDIN Corporation,4,,
2,Sep 23 2022 8:30AM,247106,10,ISDIN Corporation,10,,
3,Sep 23 2022 8:30AM,247067,10,Eywa Pharma Inc.,1,,
4,Sep 23 2022 8:30AM,247109,10,ISDIN Corporation,2,,
5,Sep 22 2022 4:47PM,247111,16,Sartorius Lab Products and Service,2,,
6,Sep 22 2022 4:47PM,247110,16,Sartorius Bioprocess Solutions,3,,
7,Sep 22 2022 4:22PM,247107,16,"Emersa Waterbox, LLC",1,,
8,Sep 22 2022 4:11PM,247103,19,Eywa Pharma Inc.,3,,
9,Sep 22 2022 3:41PM,247102,12,Hush Hush,,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 23 2022 8:30AM,247104,10,ISDIN Corporation,11,,
1,Sep 23 2022 8:30AM,247105,10,ISDIN Corporation,4,,
2,Sep 23 2022 8:30AM,247106,10,ISDIN Corporation,10,,
3,Sep 23 2022 8:30AM,247067,10,Eywa Pharma Inc.,1,,
4,Sep 23 2022 8:30AM,247109,10,ISDIN Corporation,2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 23 2022 8:30AM,247104,10,ISDIN Corporation,11.0,NaN,NaN
1,Sep 23 2022 8:30AM,247105,10,ISDIN Corporation,4.0,NaN,NaN
2,Sep 23 2022 8:30AM,247106,10,ISDIN Corporation,10.0,NaN,NaN
3,Sep 23 2022 8:30AM,247067,10,Eywa Pharma Inc.,1.0,NaN,NaN
4,Sep 23 2022 8:30AM,247109,10,ISDIN Corporation,2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 23 2022 8:30AM,247104,10,ISDIN Corporation,11.0,0.0,0.0
1,Sep 23 2022 8:30AM,247105,10,ISDIN Corporation,4.0,0.0,0.0
2,Sep 23 2022 8:30AM,247106,10,ISDIN Corporation,10.0,0.0,0.0
3,Sep 23 2022 8:30AM,247067,10,Eywa Pharma Inc.,1.0,0.0,0.0
4,Sep 23 2022 8:30AM,247109,10,ISDIN Corporation,2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1976675,72.0,11.0,10.0
12,247102,0.0,1.0,0.0
16,741328,6.0,0.0,0.0
19,988374,6.0,0.0,0.0
20,494079,35.0,1.0,0.0
21,493997,1.0,1.0,0.0
22,988336,4.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1976675,72.0,11.0,10.0
1,12,247102,0.0,1.0,0.0
2,16,741328,6.0,0.0,0.0
3,19,988374,6.0,0.0,0.0
4,20,494079,35.0,1.0,0.0
5,21,493997,1.0,1.0,0.0
6,22,988336,4.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,72.0,11.0,10.0
1,12,0.0,1.0,0.0
2,16,6.0,0.0,0.0
3,19,6.0,0.0,0.0
4,20,35.0,1.0,0.0
5,21,1.0,1.0,0.0
6,22,4.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,72.0
1,12,Released,0.0
2,16,Released,6.0
3,19,Released,6.0
4,20,Released,35.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,21,22
Status,,,,,,,
Completed,10.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,11.0,1.0,0.0,0.0,1.0,1.0,0.0
Released,72.0,0.0,6.0,6.0,35.0,1.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,21,22
0,Completed,10.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,11.0,1.0,0.0,0.0,1.0,1.0,0.0
2,Released,72.0,0.0,6.0,6.0,35.0,1.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,21,22
0,Completed,10.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,11.0,1.0,0.0,0.0,1.0,1.0,0.0
2,Released,72.0,0.0,6.0,6.0,35.0,1.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()